Se importa todo lo necesario

In [61]:
#numpy
import numpy as np

#sklearn
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity

#pyclustering
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.cluster.kmeans import kmeans
from pyclustering.utils import read_sample

#silhouette to evaluate clusterization quality
from pyclustering.cluster.silhouette import silhouette
import dataScienceUtils as ut

import pandas as pd

Se define el path

In [47]:
#define basic info
path='rawDataToFormattedData\\PorCentrosDeSalud\\formattedData.txt'

Se leen y se preprocesan los datos

In [48]:
# load list of points for cluster analysis
init_sample = read_sample(path)
number_of_data_points=len(init_sample)
#preprocessing
norm_sample=preprocessing.normalize(np.asarray(init_sample))

Se definen funciones necesarias para el algoritmo genetico

In [49]:
def get_random_individual(sample, k):
    return kmeans_plusplus_initializer(sample,k, kmeans_plusplus_initializer.FARTHEST_CENTER_CANDIDATE).initialize()

def compute_score(individual,sample):
    kmeans_instance = kmeans(sample,individual)
    kmeans_instance.process()
    clusters = kmeans_instance.get_clusters()
    dirtyscore = silhouette(sample, clusters).process().get_score()
    score=[x if str(x)!='nan' else 0 for x in dirtyscore]
    return np.mean(np.asarray(score))

def custom_sort(individual):
    return -individual[1]

def cross(individual1,individual2,sample):
    minimum_length=min(len(individual1[0]),len(individual2[0]))
    maximum_length=max(len(individual1[0]),len(individual2[0]))
    new_individual=[]
    for i in range(minimum_length):
        new_individual.append((individual1[0][i]+individual2[0][i])/2)
    for i in range(minimum_length+1,(minimum_length+maximum_length)//2):
        if len(individual1[0])==maximum_length:
            new_individual.append(individual1[0][i])
        else:
            new_individual.append(individual2[0][i])
    return (new_individual,compute_score(new_individual,sample))            

Se definen hiperparametros del algoritmo genetico

In [50]:
base_individuals=100
number_of_keis=1
number_of_clusters_k=4
lucky_individuals=10
generations=30

Se inicializa la poblacion

In [51]:
poblation=[get_random_individual(norm_sample,k)  for i in range(base_individuals) for k in range(number_of_clusters_k,number_of_keis+number_of_clusters_k)]
poblation=[(x,compute_score(x,norm_sample)) for x in poblation]
poblation.sort(key=custom_sort)

Se corre el algoritmo genetico

In [52]:
for t in range(30):
    ofspring=[]
    for i,x in enumerate(poblation):
        for j,y in enumerate(poblation):
            if i<j and i<lucky_individuals and j<lucky_individuals:
                ofspring.append(cross(x,y,norm_sample))
    for of in ofspring:
        poblation.append(of)
    poblation.sort(key=custom_sort)
    poblation=poblation[:base_individuals]

Se computan los agrupamientos del mejor individuo

In [58]:
kmeans_instance = kmeans(norm_sample,poblation[0][0])
kmeans_instance.process()
clusters = kmeans_instance.get_clusters()
print(poblation[0][1])
for clusteri in clusters:
    print(clusteri)

0.367987235333441
[1, 5, 8, 11, 12, 13, 18]
[0, 3, 4, 6, 7, 9, 10, 14, 15, 16, 17]
[19, 20, 21]
[2]


Se cargan las encuestas organizacionales por cada hospital en pandas

In [90]:
csvPath='rawDataToFormattedData\\PorCentrosDeSalud\\pandaData.csv'
hospitales=pd.read_csv(csvPath,sep=',')

In [99]:
visual_testing=np.asarray(init_sample);

In [111]:
visual_testing[5].reshape(27,7)

array([[ 0.,  0.,  2.,  4.,  3.,  2.,  4.],
       [ 0.,  0.,  0.,  1.,  6.,  5.,  3.],
       [ 0.,  1.,  0.,  3.,  2.,  6.,  3.],
       [ 0.,  0.,  2.,  3.,  6.,  2.,  2.],
       [ 2.,  0.,  2.,  4.,  6.,  1.,  0.],
       [ 3.,  0.,  1.,  1.,  7.,  3.,  0.],
       [ 0.,  0.,  2.,  1.,  6.,  4.,  2.],
       [ 0.,  0.,  3.,  4.,  6.,  1.,  1.],
       [ 0.,  1.,  3.,  2.,  5.,  4.,  0.],
       [ 2.,  2.,  5.,  1.,  3.,  2.,  0.],
       [10.,  2.,  3.,  0.,  0.,  0.,  0.],
       [ 3.,  1.,  8.,  2.,  1.,  0.,  0.],
       [ 0.,  3.,  2.,  4.,  3.,  2.,  1.],
       [ 0.,  1.,  2.,  5.,  3.,  3.,  1.],
       [11.,  2.,  0.,  1.,  1.,  0.,  0.],
       [ 4.,  1.,  6.,  2.,  1.,  1.,  0.],
       [ 2.,  2.,  0.,  4.,  4.,  3.,  0.],
       [ 1.,  3.,  4.,  3.,  3.,  1.,  0.],
       [ 5.,  2.,  6.,  0.,  2.,  0.,  0.],
       [ 1.,  3.,  7.,  1.,  1.,  1.,  1.],
       [ 3.,  2.,  5.,  1.,  1.,  3.,  0.],
       [ 8.,  4.,  3.,  0.,  0.,  0.,  0.],
       [ 2.,  1.,  4.,  5.,  3.,

In [112]:
hospitales[hospitales.ID==5]

,EVA,TD,MD,ED,NE,DA,MA,TA,HOSPITAL,ID
135,SAI1,0,0,2,4,3,2,4,CSC0,5
136,SAI2,0,0,0,1,6,5,3,CSC0,5
137,SAI3,0,1,0,3,2,6,3,CSC0,5
138,SAI4,0,0,2,3,6,2,2,CSC0,5
139,SAI5,2,0,2,4,6,1,0,CSC0,5
140,SAO1,3,0,1,1,7,3,0,CSC0,5
141,SAO2,0,0,2,1,6,4,2,CSC0,5
142,SAO3,0,0,3,4,6,1,1,CSC0,5
143,SAO4,0,1,3,2,5,4,0,CSC0,5
144,EI1,2,2,5,1,3,2,0,CSC0,5
